In [159]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import nltk.translate.bleu_score as bleu
import tensorflow as tf
from  sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import Model

In [278]:
## LOADING THE PROCESSED DATASET
df= pd.read_csv("data.csv")
df

,input,output,y
0,And he took in my favorite subject like soccer .,And he took in my favorite subjects like socce...,1
1,"Actually , who let me know about Lang - 8 was ...","Actually , he was the one who let me know abou...",1
2,His Kanji 's ability is much better than me .,His Kanji ability is much better than mine .\n,1
3,"We 've known each other for only half a year ,...","We 've known each other for only half a year ,...",1
4,I heard a sentence last night when I watched TV .,I heard a sentence last night when I was watch...,1
...,...,...,...
511158,Hmmm... Thk i usually book on wkends... Depend...,Hmm. I think I usually book on weekends. It de...,2
511159,ask them got any sms messages to gif me lei......,Can you ask them whether they have for any sms...,2
511160,We r near coca oredi...\n,We are near Coca already.\n,2
511161,hall Eleven. Got lectures le mah.èn forget abt...,Hall eleven. Got lectures. And forget about co...,2


In [279]:
def remove_spaces(text):
    text = re.sub(r" '(\w)",r"'\1",text)
    text = re.sub(r" \,",",",text)
    text = re.sub(r" \.+",".",text)
    text = re.sub(r" \!+","!",text)
    text = re.sub(r" \?+","?",text)
    text = re.sub(" n't","n't",text)
    text = re.sub("[\(\)\;\_\^\`\/]","",text)
    
    return text

In [280]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text
def clear_result(value):
    if value > 0.1:
        return value
    else:
        return 0.47

In [281]:
def preprocess(text):
    text = re.sub("\n","",text)
    text = remove_spaces(text)   # REMOVING UNWANTED SPACES
    text = re.sub(r"\.+",".",text)
    text = re.sub(r"\!+","!",text)
    text = decontract(text)    # DECONTRACTION
    text = re.sub("[^A-Za-z0-9 ]+","",text)
    text = text.lower()
    return text

In [282]:
df["processed_input"] = df.input.apply(preprocess) 
df["processed_output"] = df.output.apply(preprocess) 

In [283]:
df =df.drop(["input","output"],axis=1)

In [284]:
df = df.drop_duplicates()


In [285]:
df[["processed_input","processed_output","y"]].to_csv("My_processed_data.csv",index=False)

In [286]:
df= pd.read_csv("My_processed_data.csv")


In [287]:
df.columns = ["enc_input","dec_input","y"] 
df["dec_output"] = df.dec_input

In [288]:
df["dec_input"]= "<start> " + df["dec_input"]
df["dec_output"] =  df["dec_output"] + " <end>" 

### all data:

In [104]:
df_train ,test_data = train_test_split(df,test_size=0.4,random_state = 3, stratify = df.y)

In [121]:
# df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,y,enc_input,dec_input,dec_output
467725,1,i study esl in the morning,<start> i study eslin the morning <end>,i study eslin the morning <end>
157998,1,there are lots of fake antiques in the market,<start> there are lots of fake antiques on the...,there are lots of fake antiques on the market ...
92568,1,for example the terms herbivorous boys was i...,<start> for example the term herbivorous boys...,for example the term herbivorous boys was in...
63596,1,now i am going to say about my dream,<start> now i am going to talk about my dream,now i am going to talk about my dream <end>
137183,1,but i do not use iphone in sex until now,<start> but i have never used iphone for sex,but i have never used iphone for sex <end>
...,...,...,...,...
171848,1,he appealed for the world peace,<start> he appealed for world peace,he appealed for world peace <end>
458048,1,it s a good phenomenon to the ordinary people...,<start> it s a good phenomenon for ordinary p...,it s a good phenomenon for ordinary people li...
130664,1,i begin to learn english on today,<start> i am beginning to learn english today,i am beginning to learn english today <end>
372999,1,i am interesting in manga japan animation and ...,<start> i am interested in manga japanese anim...,i am interested in manga japanese animation an...


In [105]:
df_val,df_test = train_test_split(test_data,test_size=0.5,random_state = 3, stratify = test_data.y)

### part of the data:

In [289]:
df_sampled = pd.concat((df[df.y==1].sample(frac= 0.2,random_state=1),df[df.y==2]))

In [290]:
df_train ,df_val = train_test_split(df_sampled,test_size=0.2,random_state = 3, stratify = df_sampled.y )

In [291]:
df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [292]:
np.random.seed(5) 
df_test = df.loc[np.random.choice(np.array([x for x in df.index.values if x not in df_sampled.index.values]),1000,replace= False,)]

In [293]:
from tensorflow.keras.preprocessing.text import Tokenizer
tk_inp = Tokenizer()
tk_inp.fit_on_texts(df_train.enc_input.apply(str))

In [294]:
tk_out = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' )
tk_out.fit_on_texts(df_train.dec_input.apply(str))

In [295]:
encoder_inp = df_train["enc_input"].apply(str).values
decoder_inp = df_train["dec_input"].apply(str).values
decoder_out = df_train["dec_output"].apply(str).values

In [296]:
def process_sample(j,max_len):
    encoder_seq = tk_inp.texts_to_sequences([encoder_inp[j]])
    decoder_inp_seq = tk_out.texts_to_sequences([decoder_inp[j]])
    decoder_out_seq = tk_out.texts_to_sequences([decoder_out[j]])
        
    encoder_seq = pad_sequences(encoder_seq, padding="post",maxlen = max_len)
    decoder_inp_seq = pad_sequences(decoder_inp_seq, padding="post",maxlen = max_len)
    decoder_out_seq = pad_sequences(decoder_out_seq ,padding="post", maxlen = max_len)
    
    return encoder_seq ,  decoder_inp_seq,  decoder_out_seq

In [297]:
class ProcessAllData(tf.keras.utils.Sequence):
    def __init__(self,batch_size):
        self.batch_size = batch_size
        self.samples_count = encoder_inp.shape[0]
        
    def __getitem__(self,i):
        start = i * self.batch_size
        stop = (i+1)*self.batch_size
        
        encoder_batches =[]
        decoder_input_batches = []
        decoder_output_batches =[]
        
        for j in range(start,stop): 
            a,b,c = process_sample(j,35)
            encoder_batches.append(a[0]) 
            decoder_input_batches.append(b[0])
            decoder_output_batches.append(c[0])
        encoder_batches = (np.array(encoder_batches)) 
        decoder_input_batches = np.array(decoder_input_batches)
        decoder_output_batches = np.array(decoder_output_batches)
        
        return [encoder_batches , decoder_input_batches],decoder_output_batches
    def __len__(self):
        return int(self.samples_count/self.batch_size)

In [298]:
train_data_batches = ProcessAllData(batch_size = 512)
val_data_batches = ProcessAllData(batch_size=512)

### Encoder Implementaion:

In [299]:
embedding = layers.Embedding(input_dim= len(tk_inp.word_index)+1,
                                         output_dim = 300,
                                         mask_zero = True,
                                          input_length = 35
                                         )
lstm=layers.Bidirectional(layers.LSTM(units= 256,return_state = True,return_sequences=True ))

encoder_input = layers.Input(shape=(35))

concat1 = layers.Concatenate()
concat2 = layers.Concatenate()

embedd=embedding(encoder_input)

output , state_h1 , state_c1 ,state_h2 , state_c2 = lstm(embedd)
state_h = concat1([state_h1,state_h2])
state_c  = concat2(([state_c1,state_c2]))
encoder_state = [state_h,state_c] 


### Decoder Implementaion:

In [300]:
embedding2 = layers.Embedding(input_dim = len(tk_out.word_index)+1,
                                          output_dim = 300,
                                         mask_zero=True,
                                         input_length = 35)
lstm2 = layers.LSTM(units = 512,
                    return_sequences=True,
                    return_state=True)
decoder_input=layers.Input(shape = (35))
embedd2=embedding(decoder_input)

output2,state_h,state_c = lstm2(embedd2,initial_state = encoder_state)

#### Dense Layer

In [301]:
dense = layers.Dense(len(tk_out.word_index)+1,activation="softmax")(output2)

### Building The Model

In [302]:
model  = Model(inputs=[encoder_input,decoder_input],outputs=dense)                           

In [303]:
callback =[ tf.keras.callbacks.ModelCheckpoint( "My_Model/besh.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001),
            tf.keras.callbacks.TensorBoard("My_Model/logs/save",histogram_freq=1)
]

In [304]:
train_steps = len(train_data_batches)
validation_steps  = len(val_data_batches)

In [305]:
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy')

In [306]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           [(None, 35)]         0                                            
__________________________________________________________________________________________________
input_25 (InputLayer)           [(None, 35)]         0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 35, 300)      10013400    input_25[0][0]                   
                                                                 input_26[0][0]                   
__________________________________________________________________________________________________
bidirectional_12 (Bidirectional [(None, 35, 512), (N 1140736     embedding_18[0][0]        

In [308]:
%load Encoder.py

In [310]:
%load Decoder.py

In [312]:
%load Model_Structure

In [ ]:
history = model.fit(train_data_batches,
                    steps_per_epoch=train_steps,
                    epochs=50,
                    validation_data = val_data_batches,
                    validation_steps =validation_steps,
                    callbacks=callback)

Epoch 1/50
  2/592 [..............................] - ETA: 4:37:51 - loss: 4.0105 

In [313]:
model.load_weights("My_Best_Weights/besh.h5")

In [314]:
#From the internet
def predict(ita_text,model):
    
    seq = tk_inp.texts_to_sequences([ita_text])

    seq = pad_sequences(seq,maxlen = 20 , padding="post")
    
    enc_output,state_h,state_c= model.layers[2](seq)
    
    pred = []
    
    input_state = [state_h,state_c]
    current_vec = tf.ones((1,1))
    
    for i in range(20):
        dec_output,dec_state_h,dec_state_c = model.layers[3](current_vec , input_state)

        dense = model.layers[4](dec_output)

        current_vec = np.argmax(dense ,axis = -1)

        input_state = [dec_state_h,dec_state_c]

        pred.append(tk_out.index_word[current_vec[0][0]])
        
        if tk_out.index_word[current_vec[0][0]]=="<end>":
            break
        
    return " ".join(pred)

In [197]:
BLEU = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size =2000 ,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model).split()
        act = [str(i.dec_output).split()]
        b = bleu.sentence_bleu(act,pred)
        b=clear_result(b)
        BLEU.append(b)
    except:
      continue
print("BELU = ", np.mean(BLEU))

0it [00:00, ?it/s]C:\Users\Moaed\AppData\Roaming\Python\Python38\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
3it [00:00,  6.27it/s]C:\Users\Moaed\AppData\Roaming\Python\Python38\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\Moaed\AppData\Roaming\Python\Python38\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of 

BELU =  0.4549907036917097


### Using Beam_Search:


In [198]:
# implement from the internet
def beam_search(input,model,k):
    seq = tk_inp.texts_to_sequences([input])
    seq = pad_sequences(seq,maxlen = 35,padding="post")

    
    enc_gru ,enc_state_h,enc_state_c   = model.layers[2](seq)

    input_state = [enc_state_h,enc_state_c ]
    
    k_beams = [[tf.ones((1,1),dtype=tf.int32),0.0]]
    for i in range(35):
        candidates = []
        for sent_pred , prob in k_beams :
            if tk_out.word_index["<end>"] in sent_pred.numpy() :

                candidates += [[sent_pred , prob]]
            else:
                
                dec_gru , dec_state_h ,dec_state_c = model.layers[3](sent_pred , input_state)
                dense = model.layers[4](tf.expand_dims(dec_state_h,axis=0))
                pred = tf.argsort(dense, direction= 'DESCENDING')[:,:,:k]
                for w in range(k):
                  candidates += [[tf.concat((sent_pred, pred[:,:,w]) , axis=-1) , (prob + tf.math.log(dense[:,:,pred[:,:,w][0][0]])[0][0])]  ]
        k_beams = sorted(candidates,key=lambda tup:tup[1],reverse=True)[:k]

    all_sent = []
    for i,score in k_beams:
        sent = ""
        for j in range(1,35):
            sent +=  tk_out.index_word[i.numpy()[:,j][0]] +  " " 
            if tk_out.index_word[i.numpy()[:,j][0]] =="<end>":
                break
        all_sent.append((sent.strip(),score.numpy()))
    return all_sent

In [199]:
# VALIDATION BELU SCORE
BLEU_beam = []
np.random.seed(1)
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000,replace=False)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = beam_search(str(i.enc_input),model,3)[0][0].split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        b=clear_result(b)
        BLEU_beam.append(b)
    except:
          continue

print("BELU Beam Search Score = ",np.mean(BLEU_beam))  

2000it [34:05,  1.02s/it]

BELU Beam Search Score =  0.4578538023600538


In [200]:
test1="hi i are friends"
print(predict(test1,model))

hi i have friends <end>


In [205]:
test2="tihs is a nice place"
print(predict(test2,model))

this is a nice place <end>


In [315]:
test3 = "i am in house"
print(predict(test3,model))

i am in a house <end>


In [316]:
test4='this dog have beautiful face'
print(predict(test4,model))

this dog has a beautiful face <end>


In [317]:
test5= 'hwo are you'
print(predict(test5,model))

are you <end>


In [318]:
test6 = 'is you good'
print(predict(test6,model))

are you good <end>


In [319]:
test7 = 'is you ok'
print(predict(test7,model))

is you ok <end>
